In [97]:
dataset_version = 6

In [98]:
import os
import numpy as np
import plotly.graph_objects as go

In [99]:
dataset_path = f"C:/Users/janin/Desktop/WindVerse/datasets/"
dataset_dir = os.path.join(dataset_path, str(dataset_version))
wind_data_file = os.path.join(dataset_path, "wind/wind_data_5d_array.npz")
topology_path = os.path.join(dataset_dir, "topology", "topology_edge_index.npy")

print(f"dataset_path: {dataset_dir}")
print(f"wind_data_file: {wind_data_file}")

dataset_path: C:/Users/janin/Desktop/WindVerse/datasets/6
wind_data_file: C:/Users/janin/Desktop/WindVerse/datasets/wind/wind_data_5d_array.npz


In [100]:
wind_data = np.load(wind_data_file, allow_pickle=True)
wind_data = np.nan_to_num(wind_data["wind_data"])
n_components, n_z, n_y, n_x, n_time = wind_data.shape
print(f"✅ Wind Data Loaded: \n num of components: {n_components} \nx: {n_x}, y: {n_y}, z: {n_z}, \ntime steps: {n_time}")

✅ Wind Data Loaded: 
 num of components: 3 
x: 101, y: 101, z: 4, 
time steps: 300


In [101]:
def visualize_flag_vertices(flag, use_topology=False):

    path = os.path.join(dataset_dir, "flags")
    # 1. Load Data
    flag_path = os.path.join(path, flag)
    data = np.load(flag_path)

    edge_index = np.load(topology_path)

    # 2. Extract Vertex Positions
    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]

    # 3. Process Edges for Visualization (The "NaN" trick)
    u, v = edge_index[0], edge_index[1]

    x_lines = np.column_stack((x[u], x[v], np.full_like(u, np.nan, dtype=float))).flatten()
    y_lines = np.column_stack((y[u], y[v], np.full_like(u, np.nan, dtype=float))).flatten()
    z_lines = np.column_stack((z[u], z[v], np.full_like(u, np.nan, dtype=float))).flatten()

    # 4. Create Traces
    edge_trace = go.Scatter3d(
        x=x_lines, y=y_lines, z=z_lines,
        mode='lines',
        line=dict(color='black', width=2),
        name='Mesh Edges'
    )

    node_trace = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=4,
            color=z,
            colorscale='Viridis',
            opacity=0.8
        ),
        name='Vertices'
    )

    # 5. Define Clean Layout
    axis_style = dict(
        showbackground=False,
        showgrid=False,
        showline=True,
        linecolor='black',
        linewidth=5,
        zeroline=True,
        zerolinewidth=5,
        zerolinecolor='red',
        title_font=dict(size=20)
    )

    fig = go.Figure(data=[edge_trace, node_trace])

    fig.update_layout(
        title="Flag Topology Visualization",
        width=600, height=400,
        showlegend=False,
        scene=dict(
            xaxis={**axis_style, 'title': 'X Axis', 'linecolor': 'red'},
            yaxis={**axis_style, 'title': 'Y Axis', 'linecolor': 'green'},
            zaxis={**axis_style, 'title': 'Z Axis', 'linecolor': 'blue'},
            aspectmode='data',

            # --- CAMERA CONFIGURATION ---
            camera=dict(
                # Position of the camera (x,y,z).
                # (0, -2.5, 0.5) looks from the "Front" (negative Y) and slightly up.
                eye=dict(x=0, y=-4, z=0.5),

                # The point the camera looks at (Center of the flag usually)
                center=dict(x=0, y=0, z=0),

                # Which way is "Up" (Z-axis is up)
                up=dict(x=0, y=0, z=1)
            )
        ),
        margin=dict(r=0, b=0, l=0, t=40)
    )

    fig.show()

In [108]:
visualize_flag_vertices("flag_020_010.npy", use_topology=True)